BERT là viết tắt của Bidirectional Encoder Representations from Transformers được hiểu là một mô hình học sẵn hay còn gọi là pre-train model, học ra các vector đại diện theo ngữ cảnh 2 chiều của từ, được sử dụng để transfer sang các bài toán khác trong lĩnh vực xử lý ngôn ngữ tự nhiên.

BERT đã thành công trong việc cải thiện những công việc gần đây trong việc tìm ra đại diện của từ trong không gian số (không gian mà máy tính có thể hiểu được) thông qua ngữ cảnh của nó.

Về mặt lý thuyết, các kỹ thuật khác như Word2vec, FastText hay Glove cũng tìm ra đại diện của từ thông qua ngữ cảnh chung của chúng. Tuy nhiên, những ngữ cảnh này là đa dạng trong dữ liệu tự nhiên. Ví dụ các từ như "con chuột" có ngữ nghĩa khác nhau ở các ngữ cảnh khác nhau như "Con chuột máy tính này thật đẹp!!" và "con chuột này to thật." Trong khi các mô hình như Word2vec, fastText tìm ra 1 vector đại diện cho mỗi từ dựa trên 1 tập ngữ liệu lớn nên không thể hiện được sự đa dạng của ngữ cảnh. Việc tạo ra một biểu diễn của mỗi từ dựa trên các từ khác trong câu sẽ mang lại kết quả ý nghĩa hơn nhiều. Như trong trường hợp trên ý nghĩa của từ con chuột sẽ được biểu diễn cụ thể dựa vào phần trước hoặc sau nó trong câu. Nếu đại diện của từ "con chuột" được xây dựng dựa trên những ngữ cảnh cụ thể này thì ta sẽ có được biểu diễn tốt hơn.

BERT mở rộng khả năng của các phương pháp trước đây bằng cách tạo các biểu diễn theo ngữ cảnh dựa trên các từ trước và sau đó để dẫn đến một mô hình ngôn ngữ với ngữ nghĩa phong phú hơn.

PhoBERT là mô hình BERT được pretrained với ngữ liệu Tiếng Việt. Người dùng có thể dễ dàng finetune bằng cách load pretrained model từ thư viện transformer của Python, và huấn luyện trên bộ dữ liệu của mình để thực hiện tác vụ mong muốn.

In [1]:
import pandas as pd
import numpy as np
import torch
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from tqdm import tqdm
import re
from torch import nn
import time

In [2]:
!gdown --id 1cNE8wCT5-20Eyf-XC4QxWyp2QPRw6VnH
!gdown --id 10B6MTrlWChHQO6U6EDy-VriVUszAXp-4

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1cNE8wCT5-20Eyf-XC4QxWyp2QPRw6VnH
To: /content/train_cleaned_segmented.txt
100% 25.5M/25.5M [00:00<00:00, 93.4MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=10B6MTrlWChHQO6U6EDy-VriVUszAXp-4
To: /content/test_cleaned.txt
100% 16.4M/16.4M [00:00<00:00, 22.4MB/s]


In [2]:
train_df = pd.read_csv('train_cleaned_segmented.txt')
test_df = pd.read_csv('test_cleaned.txt')
valid_df = pd.read_csv('test_cleaned.txt')

File test có một lỗi encoding nhỏ ở dòng đầu, ô này để sửa cái đó, không cần để ý (không cần thêm vào report)

In [3]:
test_df['label'][0] = '__label__Nha_dat'
valid_df['label'][0] = '__label__Nha_dat'

In [4]:
train_df['label'].nunique()

23

BERT chỉ hoạt động được với nhãn kiểu số

Ta cần chuyển các nhãn từ kiểu string thành kiểu số bằng LabelEncoder

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_df['label'])
train_df['label'] = le.transform(train_df['label'])
test_df['label'] = le.transform(test_df['label'])
valid_df['label'] = le.transform(valid_df['label'])

Chuyển sang dạng numpy để tính toán nhanh hơn

In [5]:
sentences_train = train_df['paragraph'].to_numpy()
sentences_test = test_df['paragraph'].to_numpy()
sentences_valid = valid_df['paragraph'].to_numpy()

In [6]:
y_train = train_df['label'].to_numpy()
y_test = test_df['label'].to_numpy()
y_valid = valid_df['label'].to_numpy()

In [ ]:
y_train

array([ 4, 15, 15, ...,  9, 16,  9])

In [ ]:
!pip3 install pyarrow==15.0.2
!pip3 install datasets
!pip install --upgrade accelerate
!pip install transformers[torch]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 15.0.2 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 

Lưu dữ liệu ở dạng Hugging face dataset

In [8]:
from datasets import Dataset

# Create a DataFrame with your training data
train_data = {'text': sentences_train, 'label': y_train}
valid_data = {'text': sentences_valid, 'label': y_valid}

train_df = pd.DataFrame(train_data)
valid_df = pd.DataFrame(valid_data)

# Convert the DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

Dùng Tokenizer của PhoBERT

In [9]:
from transformers import AutoTokenizer

model_name = "vinai/phobert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)

# Remove the original text columns
train_dataset = train_dataset.remove_columns(["text"])
valid_dataset = valid_dataset.remove_columns(["text"])

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
valid_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Set the format for PyTorch/TensorFlow (here, PyTorch)
train_dataset.set_format("torch")
valid_dataset.set_format("torch")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10017 [00:00<?, ? examples/s]

Tải pretrained model PhoBERT cho phân loại văn bản

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=23)

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from datasets import load_metric

accuracy_metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    return accuracy

<ipython-input-11-0f6757ccba1f>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")


The repository for accuracy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/accuracy.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Định nghĩa các tham số

In [12]:
from transformers import TrainingArguments, Trainer
import accelerate
training_args = TrainingArguments(
    output_dir="./results_PhoBERT",
    eval_strategy ="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

Huấn luyện mô hình

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.509200,0.408895,0.868124
2,0.293200,0.337914,0.877808
3,0.236600,0.322157,0.879106


TrainOutput(global_step=3000, training_loss=0.4875901947021484, metrics={'train_runtime': 1581.8385, 'train_samples_per_second': 30.344, 'train_steps_per_second': 1.897, 'total_flos': 3157927981056000.0, 'train_loss': 0.4875901947021484, 'epoch': 3.0})

Đánh giá mô hình

In [ ]:
trainer.evaluate()

{'eval_loss': 0.3221569359302521,
 'eval_accuracy': 0.8791055206149546,
 'eval_runtime': 75.041,
 'eval_samples_per_second': 133.487,
 'eval_steps_per_second': 8.355,
 'epoch': 3.0}